In [55]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from numpy import *
from numpy.linalg import inv

In [297]:
class datasetgenerator:
    def __init__(self,size):
        self.train_arr = []
        self.size = size
        self.sigma = math.sqrt(0.1)
        self.Y = []
        self.gene()
        
    def gene(self):
        for _ in range(self.size):
            tmparr = []
            a = np.asarray([1])
            tmpfirst10 = random.normal(0,self.sigma,10)
            tmpfirst10 = np.concatenate((a, tmpfirst10), axis=None)
            for var in tmpfirst10:
                tmparr.append(var)
            tmparr.append(tmparr[0]+tmparr[1]+np.random.normal(0,self.sigma))
            tmparr.append(tmparr[2]+tmparr[3]+np.random.normal(0,self.sigma))
            tmparr.append(tmparr[3]+tmparr[4]+np.random.normal(0,self.sigma))
            tmparr.append(0.1*tmparr[6]+np.random.normal(0,self.sigma))
            tmparr.append(2*tmparr[1]-10+np.random.normal(0,self.sigma))
            # x16~x20
            tmp16 = np.random.normal(0,self.sigma,5)
            for var in tmp16:
                tmparr.append(var)
            self.train_arr.append(tmparr)
            tmpy = self.computeY(tmparr)
            self.Y.append(tmpy)
            
    def computeY(self,arr):
        tmparr = arr
        y = 0
        i = 1
        for i in range(1,11):
            y+=(0.6)**i*tmparr[i-1]
        y+=10+np.random.normal(0,self.sigma)
        return y

In [394]:
d = datasetgenerator(1000)

In [488]:
class Linear_Regression:
    def __init__(self,train_arr,Y,lamda):
        self.arr_x = np.asarray(train_arr)
        self.arr_y = np.asarray(Y)
        self.lamda = lamda
    def LSR(self):
        w = np.dot(inv(np.dot(self.arr_x.T,self.arr_x)),np.dot(self.arr_x.T,self.arr_y))
        return w
    
    def ridge(self):
        w = np.dot(inv(np.dot(self.arr_x.T,self.arr_x)+self.lamda*np.identity(21)),np.dot(self.arr_x.T,self.arr_y))
        return w
    
    

    
    def updatebias(self,w,x,y):
        sum = 0
        for i in range(len(y)):
            sum+=y[i]-np.dot(w,x[i])
    
        sum= sum/(len(y))
        w[0] = w[0]+sum
    
    def updateunbias(self,w,x,y,row,lamda):
        wi = w[row]
        #compute the big part
        tmpx = self.prunex(x,row)
        xt = tmpx.T
        numer = np.dot(xt,y-np.dot(x,w))-lamda/2
        deno = np.dot(xt,tmpx)
        
        ans1 = numer/deno
        
        numer2 =  np.dot(xt,y-np.dot(x,w))+lamda/2
        ans2 = numer2/deno
        
        #first case
        if -ans1 < wi:
            w[row]=wi+ans1
        elif wi < -ans2:
            w[row]=wi+ans2
        elif wi>=-ans2 and wi<=-ans1:
            w[row] = 0

    def Lasso(self,lamda):
        #create routine to update bias
        #routine to update unbias
        w =  np.full((21),1.0)
        for _ in range(100):
            for i in range(21):
                if i == 0:
                    self.updatebias(w,self.arr_x,self.arr_y)
                else:
                    self.updateunbias(w,self.arr_x,self.arr_y,i,lamda)
        return w
    
    
        
    def prunex(self,x,row):
        tmpx = []
        for i in range(len(x)):
            tmpx.append(x[i][row])
        tmpx = np.asarray(tmpx)
        return tmpx
                       
#     preditct section
    def predict(self,w):
        err = 0
        arr_y = np.dot(self.arr_x,w)
        for i in range(len(self.arr_y)):
            err+=abs(arr_y[i]-self.arr_y[i])**2
        return err/2
    
    
    def predict_true(self,x,y,w):
        arr_x = np.asarray(x)
        arr_y = np.asarray(y)
        err = 0
        arr_y_new = np.dot(arr_x,w)
        for i in range(len(arr_y)):
            err+=abs(arr_y_new[i]-arr_y[i])**2
        return err/2
    
    
    def ridge_predict(self,w):
        err = 0
        arr_y = np.dot(self.arr_x,w)
        for i in range(len(self.arr_y)):
            err+=abs(arr_y[i]-self.arr_y[i])**2
        err+=np.dot(w,w)*0.5*self.lamda
        return err/2
    
    def true_ridge_predict(self,x,y,w):
        arr_x = np.asarray(x)
        arr_y = np.asarray(y)
        err = 0
        arr_y_new = np.dot(arr_x,w)
        for i in range(len(arr_y)):
            err+=abs(arr_y_new[i]-arr_y[i])**2
        err+=np.dot(w,w)*0.5*self.lamda
        return err/2
    
    

In [449]:
solver = Linear_Regression(d.train_arr,d.Y,0)

# Question 1

# Generate dataset of size 1000

In [458]:
d = datasetgenerator(1000)

# Use Least Square regreesion model to fit dataset

In [472]:
solver = Linear_Regression(d.train_arr,d.Y,0)

In [473]:
m = solver.LSR()

# Out the the $\underline{w}$ after training

In [474]:
print(m)

[ 1.06107987e+01  3.49051515e-01  2.42289840e-01  1.41207074e-01
  7.07439829e-02  3.13338976e-02  3.10735656e-02  3.55039573e-02
  3.14759819e-02 -8.04133610e-03 -8.06906870e-04  9.19181565e-03
 -2.39783562e-02  7.27092811e-03 -2.12074934e-03  1.60000792e-03
 -1.50419527e-03 -1.18463064e-02 -1.98314360e-03  1.22167828e-03
 -9.55806159e-03]


Bias value is $1.06107987e+01$ which is almost equal to 10, and for weights, they are lists above from $m[1]$ to $m[20]$
For comparision, the bias is almost equal to the true bias, as for the weighs, in true weights, as $i$ increases the weight should be smaller, which is consistent with the weights that I get using least square regression.


The most significant feature is $m[0] = 1.06107987e+01$ which is the bias, the least significant feature is $m[19]$, based on the result, $\underline{w}$ could be prune by omiting $\underline{x}$ from $x_{5}$ to $x_{20}$, since the weight vector is too small on these x 

# Generate large dataset to estimate the 'true' error 
By cutting the dataset in the following ways:
80% of dataset will be used to train model, 20% dataset only used to predict and conclude true error

In [481]:
err = []
d = datasetgenerator(8000)
d2 = datasetgenerator(2000)
for _ in range(100):
    solver = Linear_Regression(d.train_arr,d.Y,0)
    m = solver.LSR()
    errtmp = solver.predict_true(d2.train_arr,d2.Y,m)
    err.append(errtmp)

In [482]:
np.mean(err)

102.28475479031786

# The mean error shown above

# Question 2

In [483]:
d = datasetgenerator(1000)

# Set $\lambda$ to be 0.5

In [489]:
solver = Linear_Regression(d.train_arr,d.Y,0.5)

In [490]:
w = solver.ridge()

# Print weight when $\lambda$ = 0.5

In [491]:
w

array([ 7.25500906e+00,  8.89002390e-01,  1.75714370e-01,  1.16726730e-01,
        4.71673539e-02,  4.01228152e-03,  6.23867122e-02, -4.23862586e-02,
       -5.20504483e-02,  2.73475515e-02, -1.50919247e-03,  6.93042122e-02,
        4.79683535e-02,  2.37696104e-02,  5.92617537e-04, -3.26268197e-01,
        2.17159121e-03,  1.73037415e-02,  4.86569446e-03, -3.39331849e-03,
       -1.01752440e-02])

# Estimate the true error by evaluating that model on a large test dataset

Generate large dataset to estimate the 'true' error
By cutting the dataset in the following ways: 80% of dataset will be used to train model, 20% dataset only used to predict and conclude true error

In [492]:
err = []
d = datasetgenerator(8000)
d2 = datasetgenerator(2000)
for _ in range(100):
    solver = Linear_Regression(d.train_arr,d.Y,0.5)
    m = solver.ridge()
    errtmp = solver.true_ridge_predict(d2.train_arr,d2.Y,m)
    err.append(errtmp)

In [493]:
np.mean(err)

113.9849253736518

# The mean error shown above

# Plot estimated true error of the ridge regression model as a funciton of $\lambda$ find optimal $\lambda$

In [538]:
lamda = 0
err = []
errlist = {}
weightlist = {}
while lamda < 5:
    d = datasetgenerator(800)
    d2 = datasetgenerator(200)
    solver = Linear_Regression(d.train_arr,d.Y,lamda)
    m = solver.ridge()
    errtmp = solver.true_ridge_predict(d2.train_arr,d2.Y,m)
    errlist[errtmp]=lamda
    err.append(errtmp)
    weightlist[lamda] = m
    lamda+=0.1

In [539]:
err.sort()

In [540]:
err[0]

9.401442995744265

In [541]:
errlist[err[0]]

0

# Based on the result, the optimal $\lambda$ is 0

# $\underline{w}$ is shown below with $\lambda = 0$

In [542]:
weightlist.get(0)

array([ 1.11955880e+01,  2.45341147e-01,  2.46216431e-01,  1.29131912e-01,
        1.04523340e-01,  2.81842569e-02, -1.69076953e-02, -2.10010207e-02,
       -1.35925217e-02,  4.35517931e-02, -1.74618239e-02, -2.63568820e-02,
       -6.63512057e-03,  7.09882434e-03,  6.40762809e-03,  5.67764819e-02,
       -1.10625013e-02,  5.79067194e-02,  2.09943501e-02,  6.02264156e-02,
        4.00006343e-03])

 The bias is $1.11955880e+01$ which is close to 10 as true bias, Also we could conclude that the computed weights are close to true weight, also the computed weights are in consistent with the true weights by showing that the weight with $i$ = 1 is largest, and weight is smaller when $i$ increases

 The Most significant feature is 10 which is bias with the $\underline{w} = 1.11955880e+01$, the least significant features is $X$ when $i=11$ to $i = 20$ That means we can prune all feature from i = 11 to i = 20

 The optimal ridge regression is same as naive least squares model since  i get smallest error by set $\lambda = 0$

# Question 3

# Show that m = 1000, as lamda increases, features are effectively eliminated from the model until all weights are set to zero

# Iterate $\lambda$ from 1 to 500

In [ ]:
lamda = 0
err = []
errlist = {}
wlist = []
lamda = 1
d = datasetgenerator(1000)
while lamda < 100:
#     d2 = datasetgenerator(200)
    solver = Linear_Regression(d.train_arr,d.Y,lamda)
    m = solver.Lasso(lamda)
    wlist.append(m)
    lamda+=1

In [ ]:
print('lamda = 0, weight is : ',wlist[0])